In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import json
import tweepy as tw
import nltk
from nltk.corpus import stopwords
from nltk import bigrams
import re
import networkx
import networkx as nx
import warnings
import numpy as np
warnings.filterwarnings("ignore")


## Dataset1

In [2]:
tweets = []
for line in open('./scamdemic.json'):
    tweets.append(json.loads(line))

print(len(tweets))
a_single_tweet = tweets[1]
a_single_tweet

2312


{'media': [{'height': 1575,
   'media_key': '3_1357408849469321217',
   'type': 'photo',
   'width': 2800},
  {'height': 2048,
   'media_key': '3_1357410713594191874',
   'type': 'photo',
   'width': 2048},
  {'height': 827,
   'media_key': '3_1357386690541846538',
   'type': 'photo',
   'width': 827},
  {'height': 827,
   'media_key': '3_1357386690533462021',
   'type': 'photo',
   'width': 827},
  {'height': 1024,
   'media_key': '3_1357410709466923017',
   'type': 'photo',
   'width': 2048},
  {'height': 140,
   'preview_image_url': 'https://pbs.twimg.com/tweet_video_thumb/EtWqy4EXUAAm8Kd.jpg',
   'media_key': '16_1357178653742944256',
   'type': 'animated_gif',
   'width': 140},
  {'height': 456,
   'preview_image_url': 'https://pbs.twimg.com/ext_tw_video_thumb/1357410641716338697/pu/img/Z7zQLRwwD5FCuAVh.jpg',
   'public_metrics': {'view_count': 4},
   'duration_ms': 94963,
   'media_key': '7_1357410641716338697',
   'type': 'video',
   'width': 826},
  {'height': 838,
   'media_ke

In [3]:
a_single_tweet.keys()

dict_keys(['media', 'users', 'tweets', 'places'])

In [4]:
print(len(a_single_tweet["media"]))
print(len(a_single_tweet["users"]))
print(len(a_single_tweet["tweets"]))
print(len(a_single_tweet["places"]))

15
204
84
1


In [18]:
a_single_tweet["tweets"]

[{'reply_settings': 'everyone',
  'lang': 'en',
  'entities': {'annotations': [{'start': 53,
     'end': 57,
     'probability': 0.5045,
     'type': 'Person',
     'normalized_text': 'COVID'},
    {'start': 121,
     'end': 128,
     'probability': 0.7532,
     'type': 'Product',
     'normalized_text': 'FaceTime'}]},
  'author_id': '198939070',
  'possibly_sensitive': False,
  'context_annotations': [{'domain': {'id': '3',
     'name': 'TV Shows',
     'description': 'Television shows from around the world'},
    'entity': {'id': '10000280083',
     'name': '60 Minutes',
     'description': "The oldest and most-watched newsmagazine on television gets the real story on America's most prevalent issues. CBS News correspondents contribute segments to each hourlong episode. Topics range from hard news coverage to politics, lifestyle, pop culture, business, health, and science. The correspondents and contributors include Sharyn Alfonsi, Anderson Cooper, Steve Kroft, Lara Logan, Norah O'Don

In [6]:
a_single_tweet["tweets"][0]['referenced_tweets'][0]

{'type': 'replied_to', 'id': '1357410701132840961'}

In [7]:
a_single_tweet["tweets"][2]

{'author_id': '51241574',
 'possibly_sensitive': False,
 'text': 'Using that break, Catholic churches, schools, offices and other entities received at least $3 billion from the paycheck program, @AP found. \n\nThat was about as much as all faith-based recipients from the four religions with the next largest amounts. \n\nhttps://t.co/4gWw4MfK6y',
 'context_annotations': [{'domain': {'id': '47',
    'name': 'Brand',
    'description': 'Brands and Companies'},
   'entity': {'id': '10040667289', 'name': 'The Associated Press'}}],
 'lang': 'en',
 'id': '1357408230088060928',
 'source': 'SocialFlow',
 'referenced_tweets': [{'type': 'replied_to', 'id': '1357408140543856648'}],
 'conversation_id': '1357407801031720962',
 'entities': {'urls': [{'start': 252,
    'end': 275,
    'url': 'https://t.co/4gWw4MfK6y',
    'expanded_url': 'http://apne.ws/6h42srX',
    'display_url': 'apne.ws/6h42srX'}],
  'mentions': [{'start': 129, 'end': 132, 'username': 'AP'}]},
 'reply_settings': 'everyone',
 'crea

In [8]:
a_single_tweet["tweets"][1]

{'author_id': '148529707',
 'possibly_sensitive': False,
 'text': 'If you think we can’t afford COVID relief, here’s your reminder that America’s billionaires could pay for every cent of working family relief in Biden’s coronavirus response plan with just their pandemic profits.',
 'context_annotations': [{'domain': {'id': '10',
    'name': 'Person',
    'description': 'Named people in the world like Nelson Mandela'},
   'entity': {'id': '1057115985256210433',
    'name': 'Robert Reich',
    'description': 'Robert Reich'}},
  {'domain': {'id': '35',
    'name': 'Politician',
    'description': 'Politicians in the world, like Joe Biden'},
   'entity': {'id': '1057115985256210433',
    'name': 'Robert Reich',
    'description': 'Robert Reich'}},
  {'domain': {'id': '123',
    'name': 'Ongoing News Story',
    'description': "Ongoing News Stories like 'Brexit'"},
   'entity': {'id': '1220701888179359745', 'name': 'COVID-19'}},
  {'domain': {'id': '10',
    'name': 'Person',
    'descripti

In [9]:
def load_reduced_tweet(tweet):
    if 'referenced_tweets' in tweet: 
        ref_tweets_type= tweet['referenced_tweets'][0]['type']
        ref_tweets_id= tweet['referenced_tweets'][0]['id']
    else:
        ref_tweets_type= ''
        ref_tweets_id= ''
    reduced_tweet = {
                        'author_id' : tweet['author_id'],
                        'id' : tweet['id'],
                        'referenced_tweets_id' : ref_tweets_id,
                        'referenced_tweets_type' : ref_tweets_type,
                        'text': tweet['text'],
                        'created_at' : tweet['created_at'],
                        'public_metrics' :tweet['public_metrics'], 
                     }
    return reduced_tweet
reduced_tweet=[]
for a_single_tweet in tweets:
    if  'tweets' in a_single_tweet: 
        for tweet in a_single_tweet["tweets"]:
            reduced_tweet.append(load_reduced_tweet(tweet))
len(reduced_tweet)

134991

In [10]:
df = pd.DataFrame(reduced_tweet)
df

,author_id,id,referenced_tweets_id,referenced_tweets_type,text,created_at,public_metrics
0,497995758,1357313110466519040,1357312243592024065,replied_to,I problemi che lascia Conte sono enormi:\n\nab...,2021-02-04T13:00:37.000Z,"{'retweet_count': 191, 'reply_count': 55, 'lik..."
1,16377377,1357385619081736194,,,🛑#ULTIMAHORA🛑 El partido político IDENTITARIOS...,2021-02-04T17:48:45.000Z,"{'retweet_count': 1, 'reply_count': 0, 'like_c..."
2,729416204,1356998998901460998,,,La fiesta de San Fermín y el infame #Runningof...,2021-02-03T16:12:27.000Z,"{'retweet_count': 37, 'reply_count': 2, 'like_..."
3,334920737,1357384236978884609,,,COVID cases have plummeted worldwide over past...,2021-02-04T17:43:15.000Z,"{'retweet_count': 49, 'reply_count': 66, 'like..."
4,252254457,1357282556698382336,,,"Es una buena noticia que, por primera vez, el ...",2021-02-04T10:59:13.000Z,"{'retweet_count': 13, 'reply_count': 3, 'like_..."
...,...,...,...,...,...,...,...
134986,341190477,1356063787233644544,,,Steven Mnuchin funneled $500B in PPP Covid rel...,2021-02-01T02:16:15.000Z,"{'retweet_count': 8648, 'reply_count': 437, 'l..."
134987,39860797,1355751395379568644,,,Reporte #Covid_19mx en el #Edoméx. https://t.c...,2021-01-31T05:34:55.000Z,"{'retweet_count': 2970, 'reply_count': 113, 'l..."
134988,108932245,1355799228300324873,,,La inmunidad humana resiste a las nuevas varia...,2021-01-31T08:45:00.000Z,"{'retweet_count': 79, 'reply_count': 0, 'like_..."
134989,77544599,1356098616985849857,1356096268771106817,replied_to,@lokiswiifts Same,2021-02-01T04:34:40.000Z,"{'retweet_count': 0, 'reply_count': 1, 'like_c..."


In [12]:
df_notnull = df[df['referenced_tweets_id'] != ""]
df_notnull 

,author_id,id,referenced_tweets_id,referenced_tweets_type,text,created_at,public_metrics
0,497995758,1357313110466519040,1357312243592024065,replied_to,I problemi che lascia Conte sono enormi:\n\nab...,2021-02-04T13:00:37.000Z,"{'retweet_count': 191, 'reply_count': 55, 'lik..."
9,28635386,1357331291792883712,1357331034120019971,replied_to,"@GovEvers Meanwhile, the Legislature has not p...",2021-02-04T14:12:52.000Z,"{'retweet_count': 15, 'reply_count': 3, 'like_..."
11,163341528,1357406408615018500,1357304575552868354,quoted,Claudia López fracasó en manejo del COVID.\n\n...,2021-02-04T19:11:21.000Z,"{'retweet_count': 51, 'reply_count': 13, 'like..."
16,2760580155,1357410040165072903,1357408636985880576,replied_to,@yarruMwerdnA @smarshallr @MrSteerpike @afneil...,2021-02-04T19:25:47.000Z,"{'retweet_count': 0, 'reply_count': 1, 'like_c..."
25,27007748,1357090617206177800,1357031031073878023,replied_to,"@mhdksafa ABSOLUTELY! (Writing from Thailand, ...",2021-02-03T22:16:31.000Z,"{'retweet_count': 30, 'reply_count': 3, 'like_..."
...,...,...,...,...,...,...,...
134955,1313128625680592896,1355956309841965057,1355930900182278144,quoted,"Briefly, Dr. ZHONG Nanshan is Chinese Fauci.\n...",2021-01-31T19:09:11.000Z,"{'retweet_count': 1427, 'reply_count': 37, 'li..."
134973,834059060244672514,1341254469690920960,1341254462132785152,replied_to,34. イタリア Dr. Roberto Petrell 医師\n\nCovid-19の意味...,2020-12-22T05:29:19.000Z,"{'retweet_count': 147, 'reply_count': 1, 'like..."
134975,252794509,1355163232739217409,1355161306358308866,quoted,Ontario has overtaken Quebec as the province w...,2021-01-29T14:37:47.000Z,"{'retweet_count': 7, 'reply_count': 1, 'like_c..."
134977,1303504618559868929,1356096613140353026,1356043156702294017,replied_to,@Llanero311 @GyliLopez @Mippcivzla @NicolasMad...,2021-02-01T04:26:42.000Z,"{'retweet_count': 15, 'reply_count': 1, 'like_..."


In [17]:
scamdemic = re.compile('.*scamdemic.*') # 
df_nonmis=df_notnull[~df_notnull['text'].str.contains(scamdemic)]


### Dataset2

In [2]:
tweets = []
for line in open('./tweets/coronavirus_smaller_aaaa.json'):
    tweets.append(json.loads(line))
print(len(tweets))
a_single_tweet = tweets[1]
a_single_tweet

60000


{'source': 'Twitter Web App',
 'text': 'Profesor Adam Wichniak: Melatonina a COVID-19\nhttps://t.co/As4UtplxnZ https://t.co/Az4iDNg5Td',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'context_annotations': [{'domain': {'id': '123',
    'name': 'Ongoing News Story',
    'description': "Ongoing News Stories like 'Brexit'"},
   'entity': {'id': '1220701888179359745', 'name': 'COVID-19'}}],
 'lang': 'pl',
 'id': '1357411152792342530',
 'author_id': '3197617796',
 'attachments': {'media_keys': ['3_1357411044575105025']},
 'created_at': '2021-02-04T19:30:12.000Z',
 'entities': {'urls': [{'start': 46,
    'end': 69,
    'url': 'https://t.co/As4UtplxnZ',
    'expanded_url': 'http://www.nfs.org.pl/index.php/melatonina-a-covid-19/',
    'display_url': 'nfs.org.pl/index.php/mela…'},
   {'start': 70,
    'end': 93,
    'url': 'https://t.co/Az4iDNg5Td',
    'expanded_url': 'https://twitter.com/irenass05/status/1357411152792342530/photo/1',
    

### Dataset3

In [13]:
tweets2 = []
for line in open('./dataset/covid-misinfo-terms-small.json'):
    tweets2.append(json.loads(line))

print(len(tweets2))
a_single_tweet2 = tweets2[0]
a_single_tweet2

170636


{'conversation_id': '1366668782677069831',
 'entities': {'mentions': [{'start': 3, 'end': 14, 'username': 'lsferguson'}]},
 'id': '1366668782677069831',
 'author_id': '126237449',
 'created_at': '2021-03-02T08:36:43.000Z',
 'lang': 'en',
 'referenced_tweets': [{'type': 'retweeted', 'id': '1366377045694029828'}],
 'reply_settings': 'everyone',
 'public_metrics': {'retweet_count': 1285,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'possibly_sensitive': False,
 'context_annotations': [{'domain': {'id': '123',
    'name': 'Ongoing News Story',
    'description': "Ongoing News Stories like 'Brexit'"},
   'entity': {'id': '1220701888179359745', 'name': 'COVID-19'}}],
 'text': 'RT @lsferguson: Hydroxychloroquine is a safe, effective drug.  It should be sold over the counter.  Used as a prophylactic, the scamdemic w…'}

In [14]:
reduced_tweets=[]
def load_tweet(tweet):
    if 'referenced_tweets' in tweet: 
            ref_tweets_type= tweet['referenced_tweets'][0]['type']
            ref_tweets_id= tweet['referenced_tweets'][0]['id']
    else:
            ref_tweets_type= ''
            ref_tweets_id= ''
    reduced_tweet = {
                        'author_id' : tweet['author_id'],
                        'id' : tweet['id'],
                        'referenced_tweets_id' : ref_tweets_id,
                        'referenced_tweets_type' : ref_tweets_type,
                        'text': tweet['text'],
                        'created_at' : tweet['created_at'],
                        'public_metrics' :tweet['public_metrics'],} 
    return reduced_tweet

for tweet in tweets2:
    if 'author_id' in tweet:
        reduced_tweets.append(load_tweet(tweet))
    else:
        if 'tweets' in tweet:
            for a_single_tweet in tweet["tweets"]:
                reduced_tweets.append(load_tweet(a_single_tweet))
len(reduced_tweets)

244449

In [15]:
df = pd.DataFrame(reduced_tweets)
df

,author_id,id,referenced_tweets_id,referenced_tweets_type,text,created_at,public_metrics
0,126237449,1366668782677069831,1366377045694029828,retweeted,"RT @lsferguson: Hydroxychloroquine is a safe, ...",2021-03-02T08:36:43.000Z,"{'retweet_count': 1285, 'reply_count': 0, 'lik..."
1,1245452098935730176,1366668731124838401,1366109545727098880,replied_to,"@GovInslee Anything helps thank you, @GovInsle...",2021-03-02T08:36:31.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
2,348795749,1366668486567731201,,,When they say the second wave of the plandemic...,2021-03-02T08:35:33.000Z,"{'retweet_count': 0, 'reply_count': 1, 'like_c..."
3,1075763979778187264,1366668198364479488,1366514380557402120,replied_to,@PaulWaddington3 @ClarkeMicah I couldnt give a...,2021-03-02T08:34:24.000Z,"{'retweet_count': 0, 'reply_count': 1, 'like_c..."
4,270416438,1366667937298472960,1366657340234231812,replied_to,@MayorofLondon @BBCNews FUCK YOUR PLANdemic. W...,2021-03-02T08:33:22.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
...,...,...,...,...,...,...,...
244444,3349241,1355667443784495107,1355662729151827972,quoted,Trump supporters are now shutting down vaccina...,2021-01-31T00:01:20.000Z,"{'retweet_count': 38, 'reply_count': 5, 'like_..."
244445,1338046991914291201,1355379200404819971,,,Decoding in English on Dr. Li-Meng Yan’s Bombs...,2021-01-30T04:55:57.000Z,"{'retweet_count': 116, 'reply_count': 0, 'like..."
244446,138182116,1355257433992671235,,,Our government is broken. \n\nOur economy is b...,2021-01-29T20:52:06.000Z,"{'retweet_count': 6701, 'reply_count': 1034, '..."
244447,3308416363,1355638583688908817,1355618128567402510,replied_to,@SenTedCruz Your party abandoned all rights to...,2021-01-30T22:06:39.000Z,"{'retweet_count': 3, 'reply_count': 4, 'like_c..."


In [16]:
df["id"].unique()
df["id"].value_counts()

1355633534346670086    230
1358237862894788609     89
1356573419958108168     79
1360998527934078976     60
1363683685388738569     58
                      ... 
1358890260575490055      1
1355989000100581379      1
1361528530882818048      1
1365401813114118147      1
1363526769559490562      1
Name: id, Length: 213110, dtype: int64

In [17]:
df_duplicationd=df.drop_duplicates(subset=['id'])

In [18]:
referenced_tweets_id = '1366377045694029828'
sourse_tweet = df_duplicationd[df_duplicationd["id"] == referenced_tweets_id]
sourse_tweet['author_id'].values.tolist()[0]

'21651982'

In [ ]:
def find_sourse_author(series):
    referenced_tweets_id = series["referenced_tweets_id"]
    if referenced_tweets_id == '':
        return None
    else:
        sourse_tweet = df_duplicationd[df_duplicationd["id"] == referenced_tweets_id]
        if sourse_tweet.empty:
            return None
        else:
            sourse_author_id = sourse_tweet['author_id'].values.tolist()[0]
        return sourse_author_id

df_duplicationd["Source_author_id"] = df_duplicationd.apply(find_sourse_author,axis=1)
df_duplicationd

In [ ]:
df_duplicationd.to_csv("./dataset/network_tweets.csv",index=False)